# Effective Precipitation - Measured as The Energy Potential of Rainfall

### Columns present across different data sources : 
- **DateValueCET**- sets the date for specific measurement
- **TimeValueCET** - sets the hour for the specific measurement

relevant columns: 
- **Value:** which is the actual measurement
- **Unit:** values(GWh), even though it is a precipitation, it is actually measured as "the energy potential associated with rainfall and water flow rather than just millimeters of rainfall. This approach quantifies how much electricity (in GWh) could be generated from the available water resources."
- **PowerPriceAreaCode:** values (BRAZIL_SOUTH, BRAZIL_NORTH, BRAZIL_NORTHEAST, RAZIL_SOUTHEAST_CENTRALWEST) + BRAZIL_SIN

In [1]:
import pandas as pd 
import sys
import os
from get_files_only import get_file_from_xdrive

# Read the Excel file
df_rain = get_file_from_xdrive('Brazil Rain Data.xlsx')
df_rain = df_rain[['DateValueCET', 'TimeValueCET', 'PowerPriceAreaCode', 'Value', 'Unit']]

display(df_rain.head())

,DateValueCET,TimeValueCET,PowerPriceAreaCode,Value,Unit
0,2024-09-09,05:00:00,BRAZIL_NORTHEAST,-178.5164,GWh
1,2024-09-09,05:00:00,BRAZIL_SOUTH,-260.3010,GWh
2,2024-09-09,05:00:00,BRAZIL_NORTH,-129.8086,GWh
3,2024-09-09,05:00:00,BRAZIL_SIN,-1050.1289,GWh
4,2024-09-09,05:00:00,BRAZIL_SOUTHEAST_CENTRALWEST,-481.5028,GWh


> NOTE: BRAZIL_SIN is not a powerprice area code, SIN is in portuguese **"National Interconnected System"**  it is a large hydrothermal system for the production and transmission of electricity, whose operation involves complex simulation models that are under the coordination and control of the National Electric System Operator - ONS, which, in turn, is inspected and regulated by the National Electric Energy Agency - ANEEL.

BRAZIL_SIN area is not neccessary for this project, as it is the aggregate of all areas and therefore it will be disregarded.

In [2]:
#Remove Brazil_SIN Area
df_rain = df_rain[(df_rain['PowerPriceAreaCode'] != 'BRAZIL_SIN')]
df_rain['DateTime'] = pd.to_datetime((df_rain['DateValueCET']).astype(str) + ' ' + (df_rain['TimeValueCET']).astype(str), format='%Y-%m-%d %H:%M:%S')
df_rain['DateTime'] = pd.to_datetime(df_rain['DateTime'], format='%Y-%m-%d %H:%M:%S')
df_rain['Month'] = df_rain['DateTime'].dt.month
#Describe the dataframe
display(df_rain.groupby(['PowerPriceAreaCode', 'Month']).describe())

Value                            \
                                     count         mean          std   
PowerPriceAreaCode           Month                                     
BRAZIL_NORTH                 1      1364.0   452.438348   574.973371   
                             2      1243.0   430.999550   554.053843   
                             3      1364.0   437.484891   526.980334   
                             4      1320.0   166.494689   419.585232   
                             5      1364.0  -114.456079   253.313873   
                             6      1320.0  -196.610940   106.106042   
                             7      1364.0  -184.959796    48.046959   
                             8      1364.0  -160.378375    70.916609   
                             9      1299.0   -57.425902   226.458528   
                             10     1333.0   167.245434   393.923721   
                             11     1290.0   375.357994   523.704688   
                             12     1333.0   451.606287   576.509475   
BRAZIL_NORTHEAST             1      1364.0   606.374069  1053.621945   
                             2      1243.0   363.832903   880.684879   
                             3      1364.0   385.074097   865.549864   
                             4      1320.0   -33.702689   502.228702   
                             5      1364.0  -200.750071   272.337691   
                             6      1320.0  -223.627847   174.056729   
                             7      1364.0  -206.407739   162.483695   
                             8      1364.0  -190.199845   179.544371   
                             9      1299.0  -105.629233   320.932526   
                             10     1333.0   136.187248   600.181646   
                             11     1290.0   550.654006   855.632376   
                             12     1333.0   728.399743   968.838181   
BRAZIL_SOUTH                 1      1364.0   258.488134   678.212605   
                             2      1243.0   231.286286   614.600677   
                             3      1364.0   114.368650   622.439442   
                             4      1320.0   168.647633   804.141153   
                             5      1364.0   275.082717   976.835309   
                             6      1320.0   284.926802   869.183478   
                             7      1364.0   250.145418   854.979407   
                             8      1364.0   156.931239   782.879805   
                             9      1299.0   331.156027   945.863873   
                             10     1333.0   370.542389  1006.886561   
                             11     1290.0   188.440127   765.018379   
                             12     1333.0   180.887509   685.661052   
BRAZIL_SOUTHEAST_CENTRALWEST 1      1364.0  2633.159502  3037.582519   
                             2      1243.0  1800.933876  2635.560481   
                             3      1364.0  1412.528423  2466.129223   
                             4      1320.0  -105.481291  1672.162772   
                             5      1364.0  -393.291162  1595.709100   
                             6      1320.0  -622.528461  1150.619360   
                             7      1364.0  -639.231301   884.752240   
                             8      1364.0  -503.584117   959.820878   
                             9      1299.0   212.524932  1744.876190   
                             10     1333.0  1050.830647  2232.739912   
                             11     1290.0  1950.236231  2596.625006   
                             12     1333.0  2637.154645  2739.587999   

                                                                        \
                                          min          25%         50%   
PowerPriceAreaCode           Month                                       
BRAZIL_NORTH                 1      -326.6698    38.335050   305.07175   
                             2      -326.2999    38.197700   315.062

## Data Visualization

In [3]:
import plotly.express as px

df_rain['DateTime'] = pd.to_datetime(df_rain['DateValueCET'].astype(str) + ' ' + df_rain['TimeValueCET'].astype(str))
df_rain['DateValueCET'] = pd.to_datetime(df_rain['DateValueCET'])

# Convert 'TimeValueCET' to hour format
df_rain['Hour'] = pd.to_datetime(df_rain['TimeValueCET'], format='%H:%M:%S').dt.hour

year_filter = 2024
df_rain_graph = df_rain[df_rain['DateValueCET'].dt.year == year_filter]

# Create a line plot
fig_1 = px.line( 
    df_rain_graph,
    x='DateTime',
    y='Value',
    color='PowerPriceAreaCode',
    title=f'Energy Potential of Rainfall in (GWh) Over Time by Region for {year_filter}',
    labels={'Rainfall Potential (GWh)': 'Rainfall Potential (GWh)', 'DateTime': 'Date and Time'}
)

""" # Update layout for better visibility
fig.update_layout(
    xaxis_title='Date and Time',
    yaxis_title='Rainfall Potential (GWh)',
    legend_title='Region',
    xaxis=dict(tickformat="%Y-%m-%d %H:%M")
) """

fig_1.show()
year_filter = 2023
df_rain_graph = df_rain[df_rain['DateValueCET'].dt.year == year_filter]

# Create a line plot
fig_2 = px.line( 
    df_rain_graph,
    x='DateTime',
    y='Value',
    color='PowerPriceAreaCode',
    title=f'Energy Potential of Rainfall in (GWh) Over Time by Region for {year_filter}',
    labels={'Rainfall Potential (GWh)': 'Rainfall Potential (GWh)', 'DateTime': 'Date and Time'}
)

""" # Update layout for better visibility
fig.update_layout(
    xaxis_title='Date and Time',
    yaxis_title='Rainfall Potential (GWh)',
    legend_title='Region',
    xaxis=dict(tickformat="%Y-%m-%d %H:%M")
) """

fig_2.show()

year_filter = 2022
df_rain_graph = df_rain[df_rain['DateValueCET'].dt.year == year_filter]

# Create a line plot
fig_3 = px.line( 
    df_rain_graph,
    x='DateTime',
    y='Value',
    color='PowerPriceAreaCode',
    title=f'Energy Potential of Rainfall in (GWh) Over Time by Region for {year_filter}',
    labels={'Rainfall Potential (GWh)': 'Rainfall Potential (GWh)', 'DateTime': 'Date and Time'}
)

""" # Update layout for better visibility
fig.update_layout(
    xaxis_title='Date and Time',
    yaxis_title='Rainfall Potential (GWh)',
    legend_title='Region',
    xaxis=dict(tickformat="%Y-%m-%d %H:%M")
) """

fig_3.show()

# Conlusion 
- The data does not follow extremely similar patterns across years, as can be seen from the graphs above. 